In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('StructStreaming').getOrCreate()
spark

In [3]:
lines = spark.readStream \
             .format("socket") \
             .option("host", "localhost") \
             .option("port", 9999) \
             .load()

NameError: name 'spark' is not defined

In [6]:
words = lines.select(F.explode(
    F.split(lines.value, " ")
).alias("word"))

In [7]:
wordCounts = words.groupBy("word").count()
type(wordCounts)

In [14]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
# Full cycle
spark.readStream
  .format("kafka")
  .option("subscribe", "input")
  .load()  ### reading IN
  .groupby('value.case("string") as key')
  .agg(count("*") as "value") ### transforming
  .writeStream()
  .format("kafka")
  .option("topic", "output") # writing OUT
